In [40]:
import pandas as pd


In [41]:
PATH = "../data/ibamco.csv"

In [48]:
df = pd.read_csv(PATH, dtype={"zipcode_iber": object, "zipcode_cac": object})
df["zipcode_iber"] = df["zipcode_iber"].str.zfill(5)
df["zipcode_cac"] = df["zipcode_cac"].str.zfill(5)
df['url'] = df.url.apply(lambda x: x.strip() if type(x) == str else x)

In [49]:
df[df.Origin.isnull()]

,media,area_of_influence,ojd,url,CCAA,PROV,AREA,platform,PGD,distribution,...,province_iber,CCAA_iber,type_media_cac,address_cac,lat_cac,lng_cac,zipcode_cac,classification_ojd,sub_classification_ojd,unique_visitors_ojd


In [50]:
import numpy as np
df['ID'] = np.arange(len(df))


In [51]:
# group all other columns, but keep the first id in grouping 

df = df[['ID','Origin', 'media', 'url', 'zipcode_iber', 'zipcode_cac']].drop_duplicates(subset=['Origin', 'media', 'url', 'zipcode_iber', 'zipcode_cac'], keep='first')
df['url'].fillna('', inplace=True)

In [52]:
df

,ID,Origin,media,url,zipcode_iber,zipcode_cac
0,0,"Iberfier, CAC, OJD",el 9 nou,el9nou.cat,08500,08500
2,2,"Iberfier, CAC, OJD",betevé,beteve.cat,08018,08099
3,3,"Iberfier, CAC, OJD",pirineus tv,pirineustv.cat,25700,25700
4,4,"Iberfier, CAC, OJD",ràdio sabadell,radiosabadell.fm,08202,08202
5,5,"AMIC, Iberfier, OJD",adolescents,adolescents.cat,08500,NaN
...,...,...,...,...,...,...
1682,1682,OJD,urgelltv,urgelltv.cat,NaN,NaN
1683,1683,OJD,21radio,21radio.cat,NaN,NaN
1684,1684,OJD,aue,aue.cat,NaN,NaN
1685,1685,OJD,fembase,fembase.cat,NaN,NaN


In [53]:
df.to_csv("../data/ibamco_clean.csv", index=False)

In [54]:
df['combine'] = df['Origin'] + "~" + df['media'] + "~" + df['url'] 

In [55]:
df = df[['ID', 'combine', 'zipcode_iber', 'zipcode_cac']]
df

,ID,combine,zipcode_iber,zipcode_cac
0,0,"Iberfier, CAC, OJD~el 9 nou~el9nou.cat",08500,08500
2,2,"Iberfier, CAC, OJD~betevé~beteve.cat",08018,08099
3,3,"Iberfier, CAC, OJD~pirineus tv~pirineustv.cat",25700,25700
4,4,"Iberfier, CAC, OJD~ràdio sabadell~radiosabadel...",08202,08202
5,5,"AMIC, Iberfier, OJD~adolescents~adolescents.cat",08500,NaN
...,...,...,...,...
1682,1682,OJD~urgelltv~urgelltv.cat,NaN,NaN
1683,1683,OJD~21radio~21radio.cat,NaN,NaN
1684,1684,OJD~aue~aue.cat,NaN,NaN
1685,1685,OJD~fembase~fembase.cat,NaN,NaN


In [56]:
combine_uniques_ids = df[['combine', 'ID']].drop_duplicates(subset='combine', keep='first')
combine_uniques_ids['zipcode_iber'] = np.nan
combine_uniques_ids['zipcode_cac'] = np.nan

In [57]:
combine_uniques_ids

,combine,ID,zipcode_iber,zipcode_cac
0,"Iberfier, CAC, OJD~el 9 nou~el9nou.cat",0,NaN,NaN
2,"Iberfier, CAC, OJD~betevé~beteve.cat",2,NaN,NaN
3,"Iberfier, CAC, OJD~pirineus tv~pirineustv.cat",3,NaN,NaN
4,"Iberfier, CAC, OJD~ràdio sabadell~radiosabadel...",4,NaN,NaN
5,"AMIC, Iberfier, OJD~adolescents~adolescents.cat",5,NaN,NaN
...,...,...,...,...
1682,OJD~urgelltv~urgelltv.cat,1682,NaN,NaN
1683,OJD~21radio~21radio.cat,1683,NaN,NaN
1684,OJD~aue~aue.cat,1684,NaN,NaN
1685,OJD~fembase~fembase.cat,1685,NaN,NaN


In [58]:
combine_uniques = df['combine'].unique()
new_df = pd.DataFrame(combine_uniques, columns=['combine'])
combine_uniques

array(['Iberfier, CAC, OJD~el 9 nou~el9nou.cat',
       'Iberfier, CAC, OJD~betevé~beteve.cat',
       'Iberfier, CAC, OJD~pirineus tv~pirineustv.cat', ...,
       'OJD~aue~aue.cat', 'OJD~fembase~fembase.cat',
       'OJD~amicib~amicib.media'], dtype=object)

In [59]:
for index, row in combine_uniques_ids.iterrows():
    rows = df[df['combine'] == row['combine']]
    z_iber = rows['zipcode_iber'].dropna().unique()
    z_cac = rows['zipcode_cac'].dropna().unique()
    media = row['combine'].split('~')[1]
    combine_uniques_ids.loc[index, 'zipcode_iber'] = ",".join(z_iber)
    combine_uniques_ids.loc[index, 'zipcode_cac'] = ",".join(z_cac)

combine_uniques_ids


/tmp/ipykernel_9243/2217180133.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '08500' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  combine_uniques_ids.loc[index, 'zipcode_iber'] = ",".join(z_iber)
/tmp/ipykernel_9243/2217180133.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '08500' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  combine_uniques_ids.loc[index, 'zipcode_cac'] = ",".join(z_cac)


,combine,ID,zipcode_iber,zipcode_cac
0,"Iberfier, CAC, OJD~el 9 nou~el9nou.cat",0,08500,08500
2,"Iberfier, CAC, OJD~betevé~beteve.cat",2,08018,08099
3,"Iberfier, CAC, OJD~pirineus tv~pirineustv.cat",3,25700,25700
4,"Iberfier, CAC, OJD~ràdio sabadell~radiosabadel...",4,08202,08202
5,"AMIC, Iberfier, OJD~adolescents~adolescents.cat",5,08500,
...,...,...,...,...
1682,OJD~urgelltv~urgelltv.cat,1682,,
1683,OJD~21radio~21radio.cat,1683,,
1684,OJD~aue~aue.cat,1684,,
1685,OJD~fembase~fembase.cat,1685,,


In [60]:
df.sort_values(by='ID')

,ID,combine,zipcode_iber,zipcode_cac
0,0,"Iberfier, CAC, OJD~el 9 nou~el9nou.cat",08500,08500
2,2,"Iberfier, CAC, OJD~betevé~beteve.cat",08018,08099
3,3,"Iberfier, CAC, OJD~pirineus tv~pirineustv.cat",25700,25700
4,4,"Iberfier, CAC, OJD~ràdio sabadell~radiosabadel...",08202,08202
5,5,"AMIC, Iberfier, OJD~adolescents~adolescents.cat",08500,NaN
...,...,...,...,...
1682,1682,OJD~urgelltv~urgelltv.cat,NaN,NaN
1683,1683,OJD~21radio~21radio.cat,NaN,NaN
1684,1684,OJD~aue~aue.cat,NaN,NaN
1685,1685,OJD~fembase~fembase.cat,NaN,NaN


In [61]:
combine_uniques_ids.sort_values(by='ID')

,combine,ID,zipcode_iber,zipcode_cac
0,"Iberfier, CAC, OJD~el 9 nou~el9nou.cat",0,08500,08500
2,"Iberfier, CAC, OJD~betevé~beteve.cat",2,08018,08099
3,"Iberfier, CAC, OJD~pirineus tv~pirineustv.cat",3,25700,25700
4,"Iberfier, CAC, OJD~ràdio sabadell~radiosabadel...",4,08202,08202
5,"AMIC, Iberfier, OJD~adolescents~adolescents.cat",5,08500,
...,...,...,...,...
1682,OJD~urgelltv~urgelltv.cat,1682,,
1683,OJD~21radio~21radio.cat,1683,,
1684,OJD~aue~aue.cat,1684,,
1685,OJD~fembase~fembase.cat,1685,,


In [62]:
combine_uniques_ids['Origin'] = combine_uniques_ids['combine'].str.split('~').str[0]
combine_uniques_ids['media'] = combine_uniques_ids['combine'].str.split('~').str[1]
combine_uniques_ids['url'] = combine_uniques_ids['combine'].str.split('~').str[2]
combine_uniques_ids


,combine,ID,zipcode_iber,zipcode_cac,Origin,media,url
0,"Iberfier, CAC, OJD~el 9 nou~el9nou.cat",0,08500,08500,"Iberfier, CAC, OJD",el 9 nou,el9nou.cat
2,"Iberfier, CAC, OJD~betevé~beteve.cat",2,08018,08099,"Iberfier, CAC, OJD",betevé,beteve.cat
3,"Iberfier, CAC, OJD~pirineus tv~pirineustv.cat",3,25700,25700,"Iberfier, CAC, OJD",pirineus tv,pirineustv.cat
4,"Iberfier, CAC, OJD~ràdio sabadell~radiosabadel...",4,08202,08202,"Iberfier, CAC, OJD",ràdio sabadell,radiosabadell.fm
5,"AMIC, Iberfier, OJD~adolescents~adolescents.cat",5,08500,,"AMIC, Iberfier, OJD",adolescents,adolescents.cat
...,...,...,...,...,...,...,...
1682,OJD~urgelltv~urgelltv.cat,1682,,,OJD,urgelltv,urgelltv.cat
1683,OJD~21radio~21radio.cat,1683,,,OJD,21radio,21radio.cat
1684,OJD~aue~aue.cat,1684,,,OJD,aue,aue.cat
1685,OJD~fembase~fembase.cat,1685,,,OJD,fembase,fembase.cat


In [63]:
combine_uniques_ids[['ID','Origin', 'media', 'url', 'zipcode_iber', 'zipcode_cac']].sort_values('ID').to_excel('ibamco_sheet.xlsx', 'main', index=False)

In [64]:
combine_uniques_ids[['ID','Origin', 'media', 'url', 'zipcode_iber', 'zipcode_cac']].sort_values('ID').to_csv('../data/ibamco_grouped2.csv', index=False)